In [ ]:
pip install requests beautifulsoup4 openai


In [ ]:
pip install openai==0.28


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 1.8 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 1.54.4
    Uninstalling openai-1.54.4:
      Successfully uninstalled openai-1.54.4


In [ ]:
# 安裝所需套件
!pip install pytesseract pillow requests beautifulsoup4 openai
!apt-get update
!apt-get install -y tesseract-ocr
!apt-get install -y tesseract-ocr-chi-tra  # 安裝繁體中文支持

In [ ]:
import requests
from bs4 import BeautifulSoup
from PIL import Image
from io import BytesIO
import pytesseract
import openai

# 設定 OpenAI API Key
openai.api_key = ""

def fetch_news_content_with_images(url):
    """
    爬取新聞內容，並根據設定提取圖片文字。
    """
    try:
        response = requests.get(url)
        response.raise_for_status()
        soup = BeautifulSoup(response.text, 'html.parser')

        # 提取文字內容
        article_content = soup.find('div', class_='content')
        text_content = article_content.get_text(strip=True) if article_content else "無法找到文章內容。"

        # 提取圖片中的文字
        image_texts = []
        if article_content:
            for img_tag in article_content.find_all('img'):
                img_url = img_tag.get('src')
                if img_url:
                    image_text = extract_text_from_image(img_url)
                    if image_text:
                        image_texts.append(image_text)

        # 合併文字和圖片中的文字
        combined_content = text_content
        if image_texts:
            combined_content += "\n\n" + "\n".join(image_texts)

        return combined_content
    except Exception as e:
        return f"爬取或解析錯誤：{e}"

def extract_text_from_image(img_url):
    """
    從圖片中提取文字
    """
    try:
        response = requests.get(img_url)
        response.raise_for_status()
        image = Image.open(BytesIO(response.content))
        # 預處理圖片並提取文字
        text = pytesseract.image_to_string(image, lang="chi_tra")  # 設定語言為繁體中文
        return text.strip()
    except Exception as e:
        return f"圖片處理錯誤：{e}"

def summarize_content(content):
    """
    使用 GPT 模型生成更結構化的摘要
    """
    try:
        # 提供更具指導性的指令
        prompt = (
            "請使用繁體中文根據以下文章內容生成摘要，摘要需包含以下部分：\n"
            "1. 主旨：用一到兩句話概述文章的核心內容。\n"
            "2. 重點：提取關鍵數字、機構名稱和時間。\n"
            "3. 詳述：分段描述文章的具體細節，適合深入了解的讀者。\n\n"
            f"文章內容如下：\n{content}"
        )
        response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            messages=[
                {"role": "system", "content": "你是一個專業的文章摘要助手，擅長生成條理清晰的摘要。"},
                {"role": "user", "content": prompt}
            ]
        )
        return response['choices'][0]['message']['content']
    except openai.error.OpenAIError as e:
        return f"API 請求錯誤：{e}"

def main():
    """
    主函式，整合新聞爬取與摘要功能
    """
    news_url = "https://ithome.com.tw/news/159391"
    print("正在抓取新聞內容和圖片文字...")
    news_content = fetch_news_content_with_images(news_url)

    if "錯誤" not in news_content and "無法" not in news_content:
        print("新聞內容抓取成功，正在進行摘要...")
        summary = summarize_content(news_content)
        print("摘要完成：")
        print(summary)
    else:
        print(news_content)

if __name__ == "__main__":
    main()


正在抓取新聞內容和圖片文字...
新聞內容抓取成功，正在進行摘要...
摘要完成：
主旨：文章講述了國泰金控及其子公司國泰世華銀行在3年內進行上雲轉型的過程與策略，強調各階段的規劃和挑戰。

重點：國泰金控在2022年的上雲計畫提前2年完成預算，計畫將全集團100套系統上雲；採用Cathay 6R雲端遷移方法論，強調公有雲與多雲混合架構；重視雲端維運成本控管(FinOps)。

詳述：文章介紹了國泰金控的上雲策略，從銀行開始發展上公有雲，並設立了4個階段的上雲規畫，包括站穩單一公有雲、發展多雲等。國泰金控也制定了一套上雲的方法論Cathay 6R，並利用雲原生技術架構推動多公雲混合雲架構的發展。文章提到國泰金控注重雲端維運成本的控管，強調了對FinOps的重視，並計劃推出跨雲監控整合平臺。此外，國泰金控也以資安需求為考量，打造了金融雲端應用環境，並累積了系統遷移、不同架構上雲下雲操作經驗。在管理上，國泰金控計劃設立委外委員會負責審查上雲委外作業，並重視法令合規、資訊安全、上雲效益和跨界人才等挑戰。文章最後指出，透過深入理解子公司對雲端的需求和使用情況，才能實現上雲效益。


In [ ]:
import requests
from bs4 import BeautifulSoup
from PIL import Image
from io import BytesIO
import pytesseract
import openai

# 設定 OpenAI API Key
openai.api_key = ""

def fetch_news_content_with_images(url):
    """
    爬取新聞內容，並根據設定提取圖片文字。
    """
    try:
        response = requests.get(url)
        response.raise_for_status()
        soup = BeautifulSoup(response.text, 'html.parser')

        # 提取文字內容
        article_content = soup.find('div', class_='content')
        text_content = article_content.get_text(strip=True) if article_content else "無法找到文章內容。"

        # 提取圖片中的文字
        image_texts = []
        if article_content:
            for img_tag in article_content.find_all('img'):
                img_url = img_tag.get('src')
                if img_url:
                    image_text = extract_text_from_image(img_url)
                    if image_text:
                        image_texts.append(image_text)

        # 合併文字和圖片中的文字
        combined_content = text_content
        if image_texts:
            combined_content += "\n\n" + "\n".join(image_texts)

        return combined_content
    except Exception as e:
        return f"爬取或解析錯誤：{e}"

def extract_text_from_image(img_url):
    """
    從圖片中提取文字
    """
    try:
        response = requests.get(img_url)
        response.raise_for_status()
        image = Image.open(BytesIO(response.content))
        # 預處理圖片並提取文字
        text = pytesseract.image_to_string(image, lang="chi_tra")  # 設定語言為繁體中文
        return text.strip()
    except Exception as e:
        return f"圖片處理錯誤：{e}"

def summarize_content(content):
    """
    使用 GPT 模型生成更結構化的摘要
    """
    try:
        # 提供更具指導性的指令
        prompt = (
            "請使用繁體中文根據以下文章內容生成摘要，摘要需包含以下部分：\n"
            "1. 主旨：用一到兩句話概述文章的核心內容。\n"
            "2. 重點：提取關鍵數字、機構名稱和時間。\n"
            "3. 詳述：分段描述文章的具體細節，適合深入了解的讀者。\n\n"
            f"文章內容如下：\n{content}"
        )
        response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            messages=[
                {"role": "system", "content": "你是一個專業的文章摘要助手，擅長生成條理清晰的摘要。"},
                {"role": "user", "content": prompt}
            ]
        )
        return response['choices'][0]['message']['content']
    except openai.error.OpenAIError as e:
        return f"API 請求錯誤：{e}"

def main():
    """
    主函式，整合新聞爬取與摘要功能
    """
    news_url = "https://ithome.com.tw/news/152373"
    print("正在抓取新聞內容和圖片文字...")
    news_content = fetch_news_content_with_images(news_url)

    if "錯誤" not in news_content and "無法" not in news_content:
        print("新聞內容抓取成功，正在進行摘要...")
        summary = summarize_content(news_content)
        print("摘要完成：")
        print(summary)
    else:
        print(news_content)

if __name__ == "__main__":
    main()


正在抓取新聞內容和圖片文字...
新聞內容抓取成功，正在進行摘要...
摘要完成：
主旨：文章關於國泰金控大規模上雲的策略和方法，包括擬定上雲策略、逐步遷移系統、建立方法論，以及導入自評系統等過程。

重點：國泰金控數數發中心將在未來3～5年內完成全集團100套系統上雲，並透過自建的Cathay 6R雲端遷移方法論來加快子公司上雲的步伐。

詳述：國泰金控集團正面臨大規模上雲的挑戰，已展開集團上雲旅程，擬定上雲策略分為四階段，包含雲環境就緒、採用、優化和應用現代化。國泰致力於建構雲端數位發展能力，具體目標是在未來3～5年完成全集團100套系統上雲。他們建立一套方法論協助子公司進行大量系統遷移，分為Rehost、Re-platform、Refactor、Rewrite、Replace和Retain六種遷移架構，以對應到IaaS、PaaS和SaaS三種不同上雲模式。此方法論還個案例的分析、系統遷移方式的設計等，以確保上雲效益。此外，國泰金控亦已自行開發了CCMA系統來協助系統上雲評估，結合6R方法論，以縮短評估時間。此舉有助於提高子公司的上雲效率，並強調採用PaaS或SaaS方式上雲，以獲得更高的雲端效益。最終，文章提及國泰金控未來將關注雲端成本控制和維運，並將Day-2 Operation維運面也納入關鍵考量。


# 參考資料

*   ChatGPT

  * https://chatgpt.com/share/675152b9-af94-8001-ad56-987bdf679133
  * https://chatgpt.com/share/675152ad-d1ec-8001-9b2b-ab793e6ef96e

*   爬取技術

  * Requests 文檔：https://docs.python-requests.org/en/latest/
 * BeautifulSoup 文檔：https://www.crummy.com/software/BeautifulSoup/

*   光學字符識別(OCR)
 * Tesseract OCR：https://github.com/tesseract-ocr/tesseract
 * Pytesseract 文檔：https://pypi.org/project/pytesseract/
*自然語言處理（NLP）與生成
 * OpenAI ChatGPT API 文檔：https://platform.openai.com/docs/